<a href="https://colab.research.google.com/github/JhonSegura25/Datos/blob/main/Red.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from sklearn import metrics

In [ ]:
# Monta el drive
drive.mount('/content/drive')

In [ ]:
# Parámetros
width_shape = 224
height_shape = 224
directory = '/content/drive/MyDrive/Univercidad/Univercidad_/DATA/'

In [ ]:
# Widgets en la barra lateral
st.sidebar.title("Parámetros")
batch_size = st.sidebar.slider("Tamaño del lote", min_value=1, max_value=64, value=32)
epochs = st.sidebar.slider("Épocas", min_value=1, max_value=50, value=10)

In [ ]:
# Carga de datos
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(directory + 'TRAIN',
                                                    target_size=(width_shape, height_shape),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_set = valid_datagen.flow_from_directory(directory + 'VALID',
                                             target_size=(width_shape, height_shape),
                                             batch_size=batch_size,
                                             class_mode='binary')

In [ ]:
 #Construcción del modelo
base_model2 = VGG16(weights="imagenet", include_top=False, input_shape=(width_shape, height_shape, 3))

# Añadir capas personalizadas
x = Flatten()(base_model2.output)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Crear el modelo completo
full_model1 = Model(inputs=base_model2.input, outputs=predictions)

# Congelar capas base
for layer in base_model2.layers:
    layer.trainable = False

# Compilar el modelo
full_model1.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

# Entrenamiento del modelo
history = full_model1.fit_generator(train_generator, validation_data=test_set, workers=10, epochs=6)

In [ ]:
# Cargar datos de prueba
test_data_dir = '/content/drive/MyDrive/Univercidad/Univercidad_/DATA/TEST'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(width_shape, height_shape),
                                                  batch_size=batch_size,
                                                  class_mode='binary',
                                                  shuffle=False)

In [ ]:
# Hacer predicciones
predictions = full_model1.predict_generator(generator=test_generator)
y_real = test_generator.classes

# Generar las etiquetas para cada predicción
y_pred = predictions[:, 0] > 0.5

In [ ]:
# Generar la matriz de confusión
matc = metrics.confusion_matrix(y_real, y_pred)

# Interfaz de Streamlit
st.title("Aplicación de Clasificación de Imágenes")
st.sidebar.title("Parámetros")

# Mostrar la matriz de confusión en la interfaz de Streamlit
st.subheader("Matriz de Confusión")
st.text("Verdaderos Positivos | Falsos Positivos")
st.text("Falsos Negativos | Verdaderos Negativos")
st.text(matc)

# Mostrar el informe de clasificación
st.subheader("Informe de Clasificación")
st.text(metrics.classification_report(y_real, y_pred, digits=4))

In [ ]:
!pip install streamlit

# Escribe tu aplicación Streamlit en un archivo .py
with open('streamlit_app.py', 'w') as f:
    f.write("""
    import streamlit as st

    def main():
        st.title("¡Hola, Streamlit en Google Colab!")
        st.write("Este es un ejemplo simple de Streamlit ejecutándose en Google Colab.")

    if __name__ == '__main__':
        main()
    """)

# Ejecuta tu aplicación Streamlit en segundo plano
!streamlit run streamlit_app.py &

# Accede a la interfaz de usuario de Streamlit
# Debería aparecer un enlace en la salida que puedes abrir haciendo clic
# o copiando y pegando en tu navegador web.
!pgrep streamlit | xargs -I {} kill -9 {}  # Detén el proceso para liberar el puerto